In [1]:
import pandas as pd
import numpy as np
import warnings
import random, math, os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../utils/")
from recsys_metric import mrr
warnings.filterwarnings('ignore')

time: 320 ms (started: 2022-06-09 14:03:47 +00:00)


In [2]:
def get_data(train_file_path, val_file_path, overview=False, concat_val=False):
    # 读取数据
    
    trn_raw_data = pd.read_csv(train_file_path, sep=',', engine='python')
    val_raw_data = pd.read_csv(val_file_path, sep=',', engine='python')
    if concat_val:
        trn_raw_data = pd.concat([trn_raw_data, val_raw_data])
    
    # 分割训练和验证集
    
    trn_data = trn_raw_data.groupby('session_id')['item_id'].apply(list).reset_index()
    val_data = val_raw_data.groupby('session_id')['item_id'].apply(list).reset_index()

    trn_user_items = {}
    val_user_items = {}
    
    # 将数组构造成字典的形式{session_id: [item_id1, item_id2,...,item_idn]}
    for session, item in zip(*(list(trn_data['session_id']), list(trn_data['item_id']))):
        trn_user_items[session] = set(item)

    for session, item in zip(*(list(val_data['session_id']), list(val_data['item_id']))):
        val_user_items[session] = set(item)
    
    if overview:
        print(f'trn_raw_data:\n {trn_raw_data} \n val_raw_data:\n {val_raw_data}')
        print('trn_data:\n',trn_data)
        print('val_data:\n', val_data)
    
    return trn_user_items, val_user_items


time: 732 µs (started: 2022-06-09 14:03:47 +00:00)


In [7]:
def Item_CF(trn_user_items, val_user_items, K, N, candidate_items=None, overview=False, prediction=False):
    '''
    trn_user_items: 表示训练数据，格式为：{user_id1: [item_id1, item_id2,...,item_idn], user_id2...}
    val_user_items: 表示验证数据，格式为：{user_id1: [item_id1, item_id2,...,item_idn], user_id2...}
    K: Ｋ表示的是相似商品的数量，为每个用户交互的每个商品都选择其最相似的K个商品
    N: N表示的是给用户推荐的商品数量，给每个用户推荐相似度最大的N个商品
    '''

    # 建立user->item的倒排表
    # 倒排表的格式为: {user_id1: [item_id1, item_id2,...,item_idn], user_id2: ...} 也就是每个用户交互过的所有商品集合
    # 由于输入的训练数据trn_user_items,本身就是这中格式的，所以这里不需要进行额外的计算
    

    # 计算商品协同过滤矩阵
    # 即利用user-items倒排表统计商品与商品之间被共同的用户交互的次数
    # 商品协同过滤矩阵的表示形式为：sim = {item_id1: {item_id２: num1, item_id3: num3,...}, item_id３: {item_id４: num２,...}, ...}
    # 商品协同过滤矩阵是一个双层的字典，用来表示商品之间共同交互的用户数量
    # 在计算商品协同过滤矩阵的同时还需要记录每个商品被多少不同用户交互的次数，其表示形式为: num = {item_id1：num1, item_id２:num2, ...}
    sim = {}#两个物品同时被某些用户喜欢的总次数(不止一个用户)
    num = {}#每个物品的总共的 被喜欢(交互)数
    print('构建相似性矩阵．．．')
    #遍历每一个user交互过的items列表，统计两个不同的item共同被user(不止一个，而是trn_user_items中的所有user)交互的总次数
    for uid, items in tqdm(trn_user_items.items()):
        #对于当前items列表中的每一个item
        for i in items:
            #统计item i 被用户交互的总次数
            if i not in num:
                num[i] = 0
            num[i] += 1
            #统计 item i 和 item j 被共同交互的总次数
            if i not in sim:
                sim[i] = {}
            for j in items:
                if j not in sim[i]:
                    sim[i][j] = 0
                if i != j:
                    sim[i][j] += 1/math.log1p(len(items) * 1.)
    
    # 计算物品的相似度矩阵
    # 商品协同过滤矩阵其实相当于是余弦相似度的分子部分,还需要除以分母,即两个商品被交互的用户数量的乘积
    # 两个商品被交互的用户数量就是上面统计的num字典
    print('计算协同过滤矩阵．．．')
    for i, items in tqdm(sim.items()):
        for j, score in items.items():
            if i != j:
                sim[i][j] = score / math.sqrt(num[i] * num[j])
                
        
    # 对验证数据中的每个用户进行TopN推荐
    # 在对用户进行推荐之前需要先通过商品相似度矩阵得到 与 当前测试用户交互过的商品最相似的前K个商品，
    # 然后对这K个用户交互的商品中除当前测试用户训练集中交互过的商品以外的商品计算最终的相似度分数
    # 最终推荐的候选商品的相似度分数是由多个相似商品对该商品分数的一个累加和
    items_rank = {}
    print('给用户进行推荐．．．')
    for uid, _ in tqdm(val_user_items.items()):
        items_rank[uid] = {} # 存储用户候选的推荐商品
        for hist_item in trn_user_items[uid]: # 遍历该用户历史喜欢的商品，用来下面寻找其相似的商品
            # 回顾：sim = {item_id1: {item_id２: num1, item_id3: num3,...}, item_id３: {item_id４: num２,...}, ...}
            # 回顾：trn_user_items = {user_id1: [item_id1, item_id2,...,item_idn], user_id2: ...} 
            #print(sim[hist_item])# {item_id２: num1, item_id3: num3,...}
            for item, score in sorted(sim[hist_item].items(), key=lambda x: x[1], reverse=True)[:K]:
                if item not in trn_user_items[uid]: # 进行推荐的商品一定不能在历史喜欢商品中出现
                    #计算当前uid对当前item的打分
                    if item not in items_rank[uid]:
                        items_rank[uid][item] = 0
                    items_rank[uid][item] += score
    
    print('为每个用户筛选出相似度分数最高的Ｎ个商品...')
    if not prediction:
        #print(items_rank)#{user1:{item1:score1,item2:score2,...},user2:{item6:score6,item9:score9,...}}
        items_rank = {k: sorted(v.items(), key=lambda x: x[1], reverse=True)[:N] for k, v in items_rank.items()}
        items_rank = {k: set([x[0] for x in v]) for k, v in items_rank.items()}
    else:
        items_rank = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in items_rank.items()}
        items_rank = {k: set([x[0] for x in v if x[0] in candidate_items][:N]) for k, v in items_rank.items()}
        
        
        
    
    if overview:
        #查看一下sim   
        for k ,v in sim.items():
            print(f"similarity[{k}]:") 
            print(v)
            break
            
    return items_rank, sim



time: 2.19 ms (started: 2022-06-09 14:09:39 +00:00)


In [8]:
def rank_itemcf(target_dict, overview=False):
    item_id_rank_arr = pd.DataFrame.from_dict(target_dict, orient='index').reset_index().rename(columns={'index': 'session_id'}).fillna(-1).astype(int)
    lack_col_num = 102 - len(item_id_rank_arr.columns)
    for i in range(100-lack_col_num,100):
        item_id_rank_arr[i]=-1
     
    if overview:
        print(f"lack_col_num:{lack_col_num}")
        display(item_id_rank_arr)
    
    res_df = pd.melt(item_id_rank_arr, id_vars=['session_id'], value_vars=range(100)).sort_values(['session_id','variable']).rename(columns={'value': 'item_id'}).drop(['variable'], axis=1)
    res_df["rank"] = (list(range(1,101))*len(target_dict))
    
    
    return res_df

time: 639 µs (started: 2022-06-09 14:09:40 +00:00)


# evaluation

In [9]:
# 读取数据
train_file_path = '../../recsys2022/train_sessions.csv'
val_file_path = '../datasets/train_last_1m.csv'
trn_user_items, val_user_items = get_data(train_file_path, val_file_path, concat_val=False)
print(f'trn_user_items len:{len(trn_user_items)},val_user_items len:{len(val_user_items)}')


# 計算ItemCF
k=3
n=100
rec_items, sim = Item_CF(trn_user_items, val_user_items, k, n, prediction=False)


# rank item & output rank result
eva_res = rank_itemcf(rec_items)

trn_user_items len:1000000,val_user_items len:81618
构建相似性矩阵．．．


100% 1000000/1000000 [00:15<00:00, 62885.95it/s]


计算协同过滤矩阵．．．


100% 23496/23496 [00:06<00:00, 3874.85it/s] 


给用户进行推荐．．．


100% 81618/81618 [03:09<00:00, 429.58it/s]


为每个用户筛选出相似度分数最高的Ｎ个商品...
time: 4min 3s (started: 2022-06-09 14:09:44 +00:00)


In [10]:
eva_label = pd.read_csv('../datasets/purchases_last_1m.csv')

mrr_score = mrr(eva_res, eva_label, overview=False)

print(mrr_score)
display(f"eva_res shape:{eva_res.shape}", eva_res.iloc[95:105,:])

0.11001448743678521


'eva_res shape:(8161800, 3)'

,session_id,item_id,rank
7753710,113,-1,96
7835328,113,-1,97
7916946,113,-1,98
7998564,113,-1,99
8080182,113,-1,100
1,115,-1,1
81619,115,-1,2
163237,115,-1,3
244855,115,-1,4
326473,115,-1,5


time: 292 ms (started: 2022-06-09 14:13:47 +00:00)


# Output

In [ ]:
# 读取数据
train_file_path = '../../recsys2022/train_sessions_purchases.csv'
val_file_path = '../../recsys2022/test_leaderboard_sessions.csv'
trn_user_items, val_user_items = get_data(train_file_path, val_file_path, concat_val=True)
print(f'trn_user_items len:{len(trn_user_items)},val_user_items len:{len(val_user_items)}')


# 計算ItemCF
candidate_items = pd.read_csv('../../recsys2022/candidate_items.csv')['item_id'].tolist()
k=5 # 4 is also acceptable  
n=100
rec_items, sim = Item_CF(trn_user_items, val_user_items, k, n, candidate_items=candidate_items, prediction=True)


# rank item & output rank result
res = rank_itemcf(rec_items)
res.to_csv("../datasets/results/leader_itemcf_k5_train_purchases_test_candidate_filter_0608.csv", index=False)

In [ ]:
# view_session_id = 113
view_session_id = 659598
print(rec_items[view_session_id])
print(val_user_items[view_session_id])
print(trn_user_items[view_session_id])
print(sim[list(trn_user_items[view_session_id])[0]])

In [ ]:
rec_items